In [ ]:
import numpy as np
from numpy.linalg import norm

from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve
from scipy.optimize import minimize_scalar

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Gross-Pitaevskii Energy Functional

In [ ]:
def grad_squared(v, h, steps):
    # Resize vector to 2D array to calculate x and y gradient
    v = v.reshape(steps, steps)
    dv_dx = np.gradient(v, h, axis=0)
    dv_dy = np.gradient(v, h, axis=1)

    # Compute |nabla v|^2
    return dv_dx * dv_dx + dv_dy * dv_dy 

# Calculate the energy of the 2D Gross-Pitaevskii equation depending on
# - The potential V
# - The repulsion constant beta
# - The current vector v (assumes 2D input as just one big vector)
# - The step size h (assumes equidistant discretization)
# - The number of steps
def calculate_energy(v, V, beta, h, steps):
    grad_sq = grad_squared(v, h, steps)
    h_sq = h * h
    v_sq = v * v
    
    kinetic = np.sum(grad_sq) * h_sq
    potential = 2 * np.sum(V * v_sq) * h_sq
    interaction = beta * np.sum(v_sq * v_sq) * h_sq

    return .25 * (kinetic + potential + interaction)

# The inverse iteration algorithm

## General inverse iteration

In [ ]:
# Runs the general inverse iteration algorithm with the following parameters
# - A which is a function depending on a vector v
# - calc_E which is a function accepting a vector and returning an energy
# Returns the following parameters
# - A boolean indicating success (True if successful)
# - The number of iterations needed (or np.inf if not successful)
# - The solution
# - An array of residuum values
# - An array of energy values
# - An array of the differences between u^n and u^{n-1} as norm(u^n - u^{n-1}
def inverse_iteration(A, calc_E, dim, max_steps, tol):
    start_vec = np.ones(dim)
    start_vec /= norm(start_vec)
    u_last = start_vec
    
    iterated_values = np.zeros((max_steps, dim))
    energy = np.zeros(max_steps)
    
    for i in range(0, max_steps):
        A_u = A(u_last)
        u_cur = spsolve(A_u, u_last)
        u_cur /= norm(u_cur)

        iterated_values[i] = u_cur
        energy[i] = calc_E(u_cur)
        
        if norm(u_cur - u_last) < tol:
            residuum = norm(iterated_values[:i] - u_cur, axis=1)
            diffs = norm(iterated_values[:i] - np.insert(iterated_values, 0, start_vec, axis=0)[:i], axis=1)
            return (True, i, u_cur, residuum, energy[:i], diffs)

        u_last = u_cur

    diffs = norm(iterated_values - np.insert(iterated_values, 0, start_vec, axis=0)[:-1], axis=1)
    return (False, np.inf, u_cur, np.array([np.inf]), energy, diffs)

## Dampened inverse iteration

In [ ]:
def compute_line(tau, u_cur, u_last):
    tmp = u_cur.T @ u_last

    # Check this to avoid an accidental blow-up
    if tmp < 1e-12:
        raise Exception("Do not want to calculate inverse: Gamma too large!")
    
    gamma = 1. / tmp
    res = (1. - tau) * u_last + tau * gamma * u_cur

    return res / norm(res)

In [ ]:
# Runs the dampened inverse iteration algorithm with adaptive dampening and turns
# off the dampening once norm(u^{n+1} - u^n) < 1e-3
# - A which is a function depending on a vector v
# - calc_E which is a function accepting a vector and returning an energy
# Returns the following parameters
# - A boolean indicating success (True if successful)
# - The number of iterations needed (or np.inf if not successful)
# - The solution
# - An array of residuum values
# - An array of energy values
# - An array of the differences between u^n and u^{n-1} as norm(u^n - u^{n-1}
def dampened_inverse_iteration(A, calc_E, dim, max_steps, tol):
    damping_stop = 1e-3
    start_vec = np.ones(dim)
    start_vec /= norm(start_vec)
    u_last = start_vec

    iterated_values = np.zeros((max_steps, dim))
    energy = np.zeros(max_steps)
    
    damping = True
    for i in range(0, max_steps):
        A_u = A(u_last)
        u_cur = spsolve(A_u, u_last)

        if damping:
            def objective(tau):
                u_tau = compute_line(tau, u_cur, u_last)
                return calc_E(u_tau)

            opt_tau = minimize_scalar(objective, bounds=(0,2), method='bounded')
            if not opt_tau.success:
                raise Exception("Could not optimize for tau!")

            tau = opt_tau.x
            print(f"Dampening chose optimal tau={tau}.")

            # compute_line is already normalized
            u_cur = compute_line(tau, u_cur, u_last)
        else:
            u_cur /= norm(u_cur)

        iterated_values[i] = u_cur
        energy[i] = calc_E(u_cur)
        
        diff = norm(u_cur - u_last)
        if diff < tol:
            # At least one more run to make sure diff < tol isn't because of small damping steps
            if damping:
                print(f"Turned off damping after {i + 1} steps because of diff < tol!")
                damping = False
            else:
                residuum = norm(iterated_values[:i] - u_cur, axis=1)
                diffs = norm(iterated_values[:i] - np.insert(iterated_values, 0, start_vec, axis=0)[:i], axis=1)
                return (True, i, u_cur, residuum, energy[:i], diffs)

        if damping and diff < damping_stop:
            print(f"Turned off damping after {i + 1} steps!")
            damping = False
        
        u_last = u_cur

    diffs = norm(iterated_values - np.insert(iterated_values, 0, start_vec, axis=0)[:-1], axis=1)
    return (False, np.inf, u_cur, np.array([np.inf]), energy, diffs)

## Inverse Iteration with Shift

In [ ]:
# Runs the inverse iteration algorithm with dynamic shifting (rayleigh-quotient of last iterated) and the following parameters
# - A which is a function depending on a vector v
# - calc_E which is a function accepting a vector and returning an energy
# Returns the following parameters
# - A boolean indicating success (True if successful)
# - The number of iterations needed (or np.inf if not successful)
# - The solution
# - An array of residuum values
# - An array of energy values
# - An array of the differences between u^n and u^{n-1} as norm(u^n - u^{n-1}
def shifted_inverse_iteration(A, calc_E, dim, max_steps, tol):
    ones = np.ones(dim)
    start_vec = ones / norm(ones)
    u_last = start_vec
    
    iterated_values = np.zeros((max_steps, dim))
    energy = np.zeros(max_steps)
    
    for i in range(0, max_steps):
        A_u = A(u_last)
        shift = u_last.T @ A_u @ u_last / (u_last.T @ u_last)
        
        u_cur = spsolve(A_u - diags(shift * ones), u_last)
        u_cur /= norm(u_cur)

        iterated_values[i] = u_cur
        energy[i] = calc_E(u_cur)
        
        if norm(u_cur - u_last) < tol:
            residuum = norm(iterated_values[:i] - u_cur, axis=1)
            diffs = norm(iterated_values[:i] - np.insert(iterated_values, 0, start_vec, axis=0)[:i], axis=1)
            return (True, i, u_cur, residuum, energy[:i], diffs)

        u_last = u_cur

    diffs = norm(iterated_values - np.insert(iterated_values, 0, start_vec, axis=0)[:-1], axis=1)
    return (False, np.inf, u_cur, np.array([np.inf]), energy, diffs)

# Solving the Gross-Pitaevskii Eigenvalue Problem

## Precomputed Matrices

In [ ]:
# Returns the 1D Laplace of size (dim_sqrt, dim_sqrt) scaled by h_inv_sq = 1/h^2
def get_D2(dim_sqrt, h_inv_sq):
    diagonals = [-np.ones(dim_sqrt-1), 2*np.ones(dim_sqrt), -np.ones(dim_sqrt-1)]
    offsets = [-1, 0, 1]
    return 1. * h_inv_sq * diags(diagonals, offsets, format='csr')

# Returns the 2D Laplace of size (dim_sqrt * dim_sqrt, dim_sqrt * dim_sqrt)
# as a sparse CSR matrix
def get_L(dim_sqrt, h_inv_sq):
    D_2 = get_D2(dim_sqrt, h_inv_sq)
    I = diags(np.ones(dim_sqrt), 0, format='csr')
    return kron(D_2, I) + kron(I, D_2)

In [ ]:
# Calculated the harmonic potential M by discretizing
#  [-bound, bound]^2 in a (steps, steps) grid
def get_M(bound, steps):
    x = np.linspace(-bound, bound, steps)
    y = np.linspace(-bound, bound, steps)
    X, Y = np.meshgrid(x, y)

    f = lambda x, y: (x * x + y * y) * .5
    Z = f(X, Y)
    return diags(Z.flatten())

In [ ]:
# Returns L_M, e.g. the constant part of the GPE (Laplace + Potential)
def get_L_M(bound, steps, h_inv_sq):
    # Steps = dim_sqrt since we discretize in a (steps, steps) field
    L_N = get_L(steps, h_inv_sq)
    M_V = get_M(bound, steps)

    return L_N + M_V

## Dynamic Matrices

In [ ]:
# Calculates the nonlinearity, e.g. the particle repulsion, depending on
# - v, the current state
# - the repulsion constant beta (nonlinearity parameter)
# - h_inv_sq = 1/h^2 where h is the step size
def get_A_squiddle(v, beta, h_inv_sq):
    # Calculate diag(|v|)^2 = diag(|v|^2) = diag(v^2)
    D = diags(np.multiply(v, v))
    return h_inv_sq * beta * D

In [ ]:
# Calculates A(v) where
# - L_M is the precomputed part of the problem
# - v is the current vector
# - beta is a nonlinearity parameter (repulsion constant)
# - h_inv_sq = 1/h^2 where h is the step size
def A_v(L_M, v, beta, h_inv_sq):
    return L_M + get_A_squiddle(v, beta, h_inv_sq)

# Plots

## Parameters

In [ ]:
max_steps = 200
tol = 1e-8
bound = 8.
steps = 200
betas = np.array([1e-1, 1e0, 1e+1, 1e+2, 1e+3])

## Precomputations

In [ ]:
h = 2 * bound / steps
h_inv_sq = 1 / (h * h)
L_M = get_L_M(bound, steps, h_inv_sq)

# Potential V. Needed seperately for dampening
V = get_M(bound, steps)

## Helper methods for visualization

In [ ]:
def plot_result(axes, taken_steps, residuum, energy, diffs, beta, color, label=None):
    pt_range = np.arange(taken_steps)
    # Only add label once - we want the legend once for the whole figure
    if label == None:
        axes[0,0].plot(pt_range, residuum[:taken_steps], label=f"beta={beta}", c=color, marker='o')
    else:
        axes[0,0].plot(pt_range, residuum[:taken_steps], label=label, c=color, marker='o')

    axes[0,1].plot(pt_range, diffs[:taken_steps], c=color, marker='o')
    axes[1,0].plot(pt_range, energy[:taken_steps], c=color, marker='o')

    pt_range = pt_range[:-1]
    energy_diff = energy[:taken_steps-1] - energy[1:taken_steps]
    axes[1,1].plot(pt_range, np.full_like(pt_range, 0), color='gray', linestyle='-.')
    axes[1,1].plot(pt_range, energy_diff, c = color, marker='o')

def plot_failed_result(axes, taken_steps, energy, diffs, beta, color):
    pt_range = np.arange(taken_steps)
    # Do not plot residuum - we don't have a result!
    
    # Only add label once - we want the legend once for the whole figure
    axes[0,1].plot(pt_range, diffs[:taken_steps], label=f"beta={beta}", c=color, marker='o')
    axes[1,0].plot(pt_range, energy[:taken_steps], c=color, marker='o')

    pt_range = pt_range[:-1]
    energy_diff = energy[:taken_steps-1] - energy[1:taken_steps]
    axes[1,1].plot(pt_range, np.full_like(pt_range, 0), color='gray', linestyle='-.')
    axes[1,1].plot(pt_range, energy_diff, c = color, marker='o')

def config_result_plot(fig, axes):
    axes[0,0].set_yscale('log')
    axes[0,0].set_xscale('log')
    axes[0,0].set_ylabel('Residuum')
    axes[0,0].set_xlabel('Steps')
    
    axes[0,1].set_yscale('log')
    axes[0,1].set_xscale('log')
    axes[0,1].set_ylabel('Differences $\Vert u^n - u^{n-1} \Vert$')
    axes[0,1].set_xlabel('Steps')
    
    axes[1,0].set_yscale('linear')
    axes[1,0].set_xscale('log')
    axes[1,0].set_ylabel('Energy $E(u^n)$')
    axes[1,0].set_xlabel('Steps')
    
    axes[1,1].set_yscale('linear')
    axes[1,1].set_xscale('log')
    axes[1,1].set_ylabel('Energy diff $E(u^{n-1}) - E(u^n)$')
    axes[1,1].set_xlabel('Steps')
    
    fig.legend()

## Helper function to run the inverse iteration algorithms and remove duplicate code

In [ ]:
# Runs one of the inverse iteration algorithms (e.g. normal, shifted, dampenened) and graphs it.
# The graph will be saved to file_name
# Algo is a function taking parameters
# - A (lambda function taking vector and returning a matrix)
# - calc_E (a lambda function to calculate the energy functional depending on a vector)
# - The dimension of the problem
# - A maximum number of steps after which to abort
# - A tolerance after which to abort
def run_algo(algo, file_name):
    colors = ["darkgreen", "gold", "orange", "red", "darkred"]

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    for i in range(betas.shape[0]):
        beta = betas[i]
        A = lambda v: A_v(L_M, v, beta, h_inv_sq)
        calc_E = lambda v: calculate_energy(v, V, beta, h, steps)
        
        print(f"Starting with bound={bound}, steps={steps} and beta={beta}")
    
        (success, taken_steps, last, residuum, energy, diffs) = algo(A, calc_E, steps * steps, max_steps, tol)
        if success:
            eigenval = last.T @ A(last) @ last / (last.T @ last)
            print(f"Successful iteration with convergence in {taken_steps + 1} steps towards eigenvalue {round(eigenval, 2)}")
            plot_result(axes, taken_steps, residuum, energy, diffs, beta, colors[i])        
        else:
            print(f"Iteration failed after {max_steps} steps!")
            plot_failed_result(axes, max_steps, energy, diffs, beta, 'black')        
    
    config_result_plot(fig, axes)
    plt.tight_layout()
    plt.savefig(file_name, dpi=300)
    plt.show()    

## Testing

In [ ]:
run_algo(inverse_iteration, 'gpe-inverse-iteration.png')

In [ ]:
run_algo(dampened_inverse_iteration, 'gpe-inverse-iteration-dampened.png')

In [ ]:
run_algo(shifted_inverse_iteration, 'gpe-inverse-iteration-shifted.png')

## Comparision dampened - undampened

In [ ]:
d_colors = ["darkgreen", "forestgreen", "green", "limegreen", "springgreen"]
u_colors = ["gold", "orange", "darkorange", "red", "darkred"]

fig, axes = plt.subplots(3, 2, figsize=(12, 12))

# A bit hacky, but it allows for reusing existing methods
u_axes = axes[[0,2],:]
d_axes = axes[1:,:]
for i in range(betas.shape[0]):
    beta = betas[i]
    A = lambda v: A_v(L_M, v, beta, h_inv_sq)
    calc_E = lambda v: calculate_energy(v, V, beta, h, steps)
    
    print(f"Starting with bound={bound}, steps={steps} and beta={beta}")

    (u_success, u_taken_steps, u_last, u_residuum, u_energy, u_diffs) = inverse_iteration(A, calc_E, steps * steps, max_steps, tol)
    (d_success, d_taken_steps, d_last, d_residuum, d_energy, d_diffs) = dampened_inverse_iteration(A, calc_E, steps * steps, max_steps, tol)

    if u_success and d_success:
        print(f"Normal iteration converged after {u_taken_steps + 1} while dampened version took {d_taken_steps + 1}!")
        print(f"Diff normal/damped: {norm(u_last - d_last)}")
        plot_result(u_axes, u_taken_steps, u_residuum, u_energy, u_diffs, beta, u_colors[i], label=f"beta={beta} (normal)")
        plot_result(d_axes, d_taken_steps, d_residuum, d_energy, d_diffs, beta, d_colors[i], label=f"beta={beta} (dampened)")
    else:
        print(f"Dampened (success={d_success}) or normal (success={u_success}) inverse iteration failed!")

config_result_plot(fig, u_axes)
config_result_plot(fig, d_axes)
plt.tight_layout()
plt.savefig('gpe-inverse-iteration-comparision.png', dpi=300)
plt.show()